In [26]:
import pandas as pd
from unidecode import unidecode

# Plots
import plotly
import plotly.graph_objects as go
import chart_studio.plotly as py
plotly.offline.init_notebook_mode(connected=True)

In [2]:
# Primera dificultad: Descargar solo la data util. 
# El dataset completo tiene muchos datos que no sirven para resolver este desafío

In [3]:
US_CODE = "us" # Extracted from Newyork example
UK_CODE = "gb" # Extracted from London example

In [4]:
cities = pd.read_csv("worldcitiespop.csv", sep=";")
display(cities.head(5))

,Country,City,AccentCity,Region,Population,Latitude,Longitude,geopoint
0,us,lost creek,Lost Creek,KY,NaN,37.481667,-83.325278,"37.4816667,-83.3252778"
1,us,edgewood village,Edgewood Village,LA,NaN,32.400556,-93.842500,"32.4005556,-93.8425"
2,us,trout dale,Trout Dale,VA,NaN,36.701944,-81.440556,"36.7019444,-81.4405556"
3,us,trivoli,Trivoli,IL,NaN,40.690556,-89.891944,"40.6905556,-89.8919444"
4,us,barhamsville,Barhamsville,VA,NaN,37.454444,-76.840556,"37.4544444,-76.8405556"


In [6]:
# Cuantas ciudades hay en US y UK?

# US: 141.982
# UK: 17708

cities["Country"].value_counts()

us    141982
gb     17708
Name: Country, dtype: int64

In [7]:
# Separamos el dataset en USA y UK

USA = cities[cities["Country"] == "us"]
UK = cities[cities["Country"] == "gb"]

In [8]:
# Para poder cruzar nombres de ciudades aplicamos un pequeño proceso de estandarizacion,4
# que consiste en pasar todo a minúscula, eliminar los espacios al comienzo y final y remover caracteres especiales
USA["processed_city"] = USA["City"].apply(lambda x: unidecode(x.lower().strip()))
UK["processed_city"] = UK["City"].apply(lambda x: unidecode(x.lower().strip()))

<ipython-input-8-edf87a0f8e0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  USA["processed_city"] = USA["City"].apply(lambda x: unidecode(x.lower().strip()))
<ipython-input-8-edf87a0f8e0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UK["processed_city"] = UK["City"].apply(lambda x: unidecode(x.lower().strip()))


In [10]:
# Cómo queremos saber que nombres de ciudades de UK existen en USA, hay que eliminar los nombres de 
# ciudades duplicados en UK

UK = UK.drop_duplicates(subset=["processed_city"])

# Esto no aplica para USA por que importa la ubicación

In [14]:
# Que ciudades de UK también existen en USA?
# Hay 9.533 ciudades de UK que existen en USA
CITIES_UK_USA = UK.merge(USA, how="inner", on="processed_city", suffixes=("_uk","_usa"))
display(CITIES_UK_USA.head(2))
print("Ciudades compartidas: ", CITIES_UK_USA.shape)

,Country_uk,City_uk,AccentCity_uk,Region_uk,Population_uk,Latitude_uk,Longitude_uk,geopoint_uk,processed_city,Country_usa,City_usa,AccentCity_usa,Region_usa,Population_usa,Latitude_usa,Longitude_usa,geopoint_usa
0,gb,weston,Weston,J7,NaN,53.916667,-1.733333,"53.916667,-1.733333",weston,us,weston,Weston,OH,NaN,41.344722,-83.797222,"41.3447222,-83.7972222"
1,gb,weston,Weston,J7,NaN,53.916667,-1.733333,"53.916667,-1.733333",weston,us,weston,Weston,LA,NaN,32.243889,-92.613333,"32.2438889,-92.6133333"


Ciudades compartidas:  (9533, 17)


In [27]:
fig = go.Figure(data=go.Scattergeo(
        lon = CITIES_UK_USA['Longitude_usa'],
        lat = CITIES_UK_USA['Latitude_usa'],
        text = CITIES_UK_USA['City_usa'],
        ))

fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo_scope='usa',
    )
fig.show()

In [ ]:
# Segunda dificultad: Cuál es la costa este y oeste de USA?